# Applying python-docx to ZMS content

In [12]:
import ZODB
import os
from Products.Five.browser.tests.pages import SimpleView
from Products.zms import standard
from Products.zms import rest_api
import json

# os.getcwd()

In [6]:
# Create a ZODB connection to an existing ZODB database file
try:
	wd = '/home/zope/instance/zms5_dev/var/'
	db = ZODB.DB(os.path.join(wd, 'Data.fs'))
	conn = db.open()
	root = conn.root
	###{'Application': <Application at >}
except:
	db.close()
	print('Error: Database connection had to be closed before reopened.')

In [7]:
# Zope/src/Testing/makerequest.py
def makerequest(app, stdout=None, environ=None):
    from io import BytesIO
    from ZPublisher.BaseRequest import RequestContainer
    from ZPublisher.HTTPRequest import HTTPRequest
    from ZPublisher.HTTPResponse import HTTPResponse

    if stdout is None:
        stdout = BytesIO()
    if environ is None:
        environ = {}
    resp = HTTPResponse(stdout=stdout)
    environ.setdefault('SERVER_NAME', 'nohost')
    environ.setdefault('SERVER_PORT', '80')
    environ.setdefault('REQUEST_METHOD', 'GET')
    req = HTTPRequest(BytesIO(), environ, resp)
    req._steps = ['noobject']  # Fake a published object.
    req['ACTUAL_URL'] = req.get('URL')  # Zope 2.7.4

    # Set default skin so that the request is usable for view look-ups.
    from zope.publisher.browser import setDefaultSkin
    setDefaultSkin(req)

    requestcontainer = RequestContainer(REQUEST=req)
    return app.__of__(requestcontainer)

In [8]:
def makeresponse():
    from io import StringIO
    from ZPublisher.HTTPResponse import HTTPResponse
    stdout = StringIO()
    resp = HTTPResponse(stdout=stdout)
    return resp

In [13]:
# ZMS-Node /myzms2/content
context = root.Application.myzms2.content 
# ##############################################
# Add REQUEST to zmscontext object
context = makerequest(context)
# Add REQUEST vars
context.REQUEST.set('lang','ger')
context.REQUEST.set('path_to_handle','')
# Add RESPONSE
context.REQUEST.set('RESPONSE', makeresponse())
# ##############################################
zmscontext = context.e5
request = rest_api._get_request(zmscontext)

# tree_nodes = rest_api.RestApiController(context,request).list_tree_nodes(zmscontext)[0:1]
# print(json.dumps(tree_nodes,indent=2))

doc = rest_api.get_attrs(zmscontext)
print(json.dumps(doc,indent=2))

{
  "id": "e5",
  "meta_id": "ZMSFolder",
  "uid": "uid:60b30ef7-4e7d-4926-9309-c454906f6957",
  "getPath": "/myzms2/content/e5",
  "active": 1,
  "title": "News about the ZMS concept",
  "titlealt": "News",
  "is_page": true,
  "is_page_element": false,
  "index_html": "./../../../../../../../myzms2/content/e5",
  "parent_uid": "uid:4b46b796-d146-43c1-8eca-954d1ba2aafc",
  "home_id": "myzms2",
  "level": 1,
  "restricted": false,
  "titleimage": null,
  "levelnfc": "",
  "attr_dc_description": "Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusa",
  "attr_dc_subject": "",
  "attr_dc_type": "",
  "attr_dc_creator": ""
}


In [11]:
from docx import Document

document = Document()

title = doc.get('title')
document.add_heading(title, 0)

text = doc.get('attr_dc_description')
document.add_paragraph(text)

document.save('demo.docx')

In [40]:
# Finally close ZODB connection
db.close()